In [1]:
import os
import yaml
import sys

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

In [2]:
from dask.distributed import Client, progress
client = Client()
client

/Users/bcarte18/anaconda3/envs/po_pandas/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:54059 Dashboard: http://127.0.0.1:54060/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [3]:
import dask.dataframe as dd

In [12]:
%%time

dc_taxi_file = os.path.join(data_path, cfg['data']['dc_taxi_2017'])

# Does not work straight off, get type errors 

dc_taxi_2017 = dd.read_csv(dc_taxi_file, 
                          dtype ={'DestinationZip':'object',
                                  'PaymentType':'float64',
                                  'Type':'object',
                                  'DestinationStreetNumber':'object',
                                  'OriginStreetNumber':'object',
                                  'OriginZip':'object'})

CPU times: user 112 ms, sys: 32.1 ms, total: 144 ms
Wall time: 163 ms


In [13]:
%%time

dc_taxi_2017 = client.persist(dc_taxi_2017)
progress(dc_taxi_2017)

CPU times: user 55.1 ms, sys: 19.7 ms, total: 74.7 ms
Wall time: 91.7 ms


VBox()

In [14]:
len(dc_taxi_2017)

11937645

In [21]:
pos_tips = dc_taxi_2017[(dc_taxi_2017.GratuityAmount > 0) & (dc_taxi_2017.FareAmount > 0)]

In [22]:
pos_tips['tip_fraction'] = pos_tips.GratuityAmount / pos_tips.FareAmount

In [20]:
dc_taxi_2017.columns

Index(['Type', 'PROVIDER NAME', 'StartDateTime', 'DateCreated', 'ID',
       'ExternalID', 'FareAmount', 'GratuityAmount', 'SurchargeAmount',
       'ExtraFareAmount', 'TollAmount', 'TotalAmount', 'PaymentType',
       'StartDateTime.1', 'EndDateTime', 'OriginStreetNumber',
       'OriginStreetName', 'OriginCity', 'OriginState', 'OriginZip',
       'OriginLatitude', 'OriginLongitude', 'DestinationStreetNumber',
       'DestinationStreetName', 'DestinationCity', 'DestinationState',
       'DestinationZip', 'DestinationLatitude', 'DestinationLongitude',
       'Milage', 'Duration', 'Unnamed: 31'],
      dtype='object')

In [25]:
dayofweek = pos_tips.groupby(pos_tips.DestinationZip).tip_fraction.mean()

In [27]:
dayofweek.head()

DestinationZip
-----    0.233220
00000    0.228730
00001    0.295140
00002    0.212737
00003    0.212170
Name: tip_fraction, dtype: float64

In [28]:
client.close()

In [29]:
client.shutdown()

/Users/bcarte18/anaconda3/envs/po_pandas/lib/python3.7/site-packages/distributed/client.py:1343: UserWarning: Shutdown is deprecated.  Please use close instead
  warnings.warn("Shutdown is deprecated.  Please use close instead")
